In [ ]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.tree_node import NodeRef
from evaluator import BenchViewer, Evaluator
from debugger.debugger import Debugger3
from evaluator import DebuggerStatus
from benchmark_consts import *
from utils.system_utils import list_smt2_files, get_name_hash

In [ ]:
qs = UNSTABLE_MARIPOSA
mv = BenchViewer(qs)
# vv = BenchViewer(UNSTABLE_VERUS)

[ERROR] no such project group singleton_f6db5998ec 


[WARN] [eval] e695dc7eeb tested count 1159 < registered 1160 
[WARN] [eval] cc5b8bed53 tested count 289 < registered 497 
[WARN] [eval] 0e45874c62 tested count 991 < registered 993 


[ERROR] no such project group singleton_0b72dcef64 
[ERROR] no such project group singleton_90e97b47ee 


[WARN] [eval] d8cf640f87 tested count 1043 < registered 1044 
[WARN] [eval] be808dd0c3 tested count 997 < registered 998 

[WARN] [eval] 203627efb4 tested count 6 < registered 404 [WARN] [eval] 9b9e796545 tested count 1076 < registered 1079 


[ERROR] no such project group singleton_d874a82c3a 


[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 
[WARN] [proof] no proof available 
[WARN] [init] previous debugging attempt has failed, run with --retry-failed if needed! 

[WARN] [proof] no proof available 

[ERROR] no such project group singleton_b1b86518f3 
[ERROR] no such project group singleton_ee8d1ab295 
[ERROR] no such project group singleton_e93855aadd 
[ERROR] no such project group singleton_ab52dadac1 
[ERROR] no such project group singleton_563f5abfa4 


[WARN] [eval] 519fdc89c1 tested count 506 < registered 508 


In [3]:
mv.status.print_status()

| category                                      |   count | percentage   |
|-----------------------------------------------|---------|--------------|
| DebuggerStatus.FINISHED                       |     523 | 95.96 %      |
| DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED |       9 | 1.65 %       |
| DebuggerStatus.SINGLETON_CREATION_FAILED      |       9 | 1.65 %       |
| DebuggerStatus.NO_PROOF                       |       4 | 0.73 %       |
| total                                         |     545 | 100.00 %     |


In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]:
    print("./src/debugger3.py --create-singleton -i", q)

for q in mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED]:
    print("./src/debugger3.py --create-singleton -i", q)

In [ ]:
for q in mv.status[DebuggerStatus.SINGLETON_NOT_RAN]:
# for q in mv.status[DebuggerStatus.SINGLETON_TESTED_EMPTY]:
    assert len(list_smt2_files(mv[q].singleton_dir)) != 0
    print("./src/exper_wizard.py manager -e verify --total-parts 12 -i ", mv[q].singleton_dir, "--clear")
    print("./src/analysis_wizard.py singleton -e verify -s z3_4_13_0 -i ", mv[q].singleton_dir)
    print("python3 src/carve_and_rerun.py", mv[q].filtered_dir)

In [ ]:
import numpy as np
# print(len(mv.fixable)/len(mv.status[DebuggerStatus.FINISHED]))

bad_min_ranks = dict()
min_ranks = []

for q in mv.fixable:
    r = mv.reviewers[q]
    report = r.build_report()
    indices = report.freq.loc[report.freq["qname"].isin(report.stabilized["qname"])].index
    report.freq["rank"] = report.freq["trace_count"].rank(method='min', ascending=False)
    min_rank = report.freq.loc[indices]["rank"].min()
    max_rank = report.freq.loc[indices]["rank"].max()
    min_ranks.append(min_rank)
    if min_rank <= 10:
        continue
    bad_min_ranks[q] = (min_rank, len(indices))

min_ranks = np.array(min_ranks)

for q, (rank, feasible) in bad_min_ranks.items():
    print(int(rank), feasible, q)

In [ ]:
def fmt_ratio(x, y):
    return f"({round(x / y * 100, 1)}%)"

def fmt_ratio_2(x, y):
    return f"{x}/{y} ({round(x / y * 100, 1)}%)"

def count_leq(r, n):
    return np.where(r <= n)[0].shape[0]

fixable_count = len(mv.fixable)
finished_count = len(mv.status[DebuggerStatus.FINISHED])
bench_total = mv.status.total
print("finished", fmt_ratio_2(finished_count, bench_total))
print("\tfixable", fmt_ratio_2(fixable_count, finished_count))
rcount = count_leq(min_ranks, 1)
print("\t\t", "top-1 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 3)
print("\t\t", "top-3 ", fmt_ratio_2(rcount, fixable_count))
rcount = count_leq(min_ranks, 10)
print("\t\t", "top-10", fmt_ratio_2(rcount, fixable_count))
print("\tno-fixes", fmt_ratio_2(finished_count - fixable_count, finished_count))
no_proof = len(mv.status[DebuggerStatus.NO_PROOF])
print("no-proof", fmt_ratio_2(no_proof, bench_total))
parse_fail = len(mv.status[DebuggerStatus.SINGLETON_CREATION_UNATTEMPTED]) + len(mv.status[DebuggerStatus.SINGLETON_CREATION_FAILED])
print("parse-fail", fmt_ratio_2(parse_fail, bench_total))
in_progress = bench_total - finished_count - parse_fail - no_proof
print("in-progress", fmt_ratio_2(in_progress, bench_total))

In [ ]:
for q in mv.status[DebuggerStatus.NO_PROOF]:
    print('"' + mv[q].name_hash + '"', end=", ")

In [ ]:
dirs_to_send = []
additional = ""

for q in mv.fixable:
    r = mv.reviewers[q]
    if r.name_hash in LIKELY_FIXED:
        continue
    r.collect_garbage()
    dirs_to_send += r.get_dirs_to_sync()
    additional += f"\"{r.name_hash}\",\n"

print(additional)

for i, d in enumerate(dirs_to_send):
    if i % 5 == 0:
        print("echo '", (i //5 +1), "/", len(dirs_to_send)//5, "'")
    print("rsync -avz", d, "g2001:/home/yizhou7/mariposa/", "--inplace", "--delete", "--relative")

In [ ]:
from utils.analysis_utils import Categorizer

f_modes = Categorizer()

for query in mv.unfixable:
    rev: Evaluator = mv[query]
    mi = rev.get_trace_info()
    reason = mi.get_failed_reason()
    f_modes.add_item(reason, query)

f_modes.finalize()

In [5]:
f_modes.print_status()

| category                  |   count | percentage   |
|---------------------------|---------|--------------|
| TraceFailure.TIMEOUT      |     116 | 82.27 %      |
| TraceFailure.FAST_UNKNOWN |      22 | 15.6 %       |
| TraceFailure.SLOW_UNKNOWN |       3 | 2.13 %       |
| total                     |     141 | 100.00 %     |
